In [1]:
!pip install --upgrade pip
!pip install openai-whisper


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 22.3 MB/s eta 0:00:0000:010:01
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 14.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.1/253.1 MB 90.0 MB/s eta 0:00:00:00:0100:01
  Created wheel for openai-whisper: filename=openai_whisper-20240930-py3-none-any.whl size=803405 sha256=af41e8332c8c5bebe1051429136020690e2fc34ffd139becec28d7bb8e616ce9
  Stored in directory: /root/.cache/pip/wheels/dd/4a/1f/d1c4bf3b9133c8168fe617ed979cab7b14fe381d059ffb9d83
Successfully built openai-whisper


In [4]:
import os
import librosa
import numpy as np
import soundfile as sf
import whisper
import torch
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
import pandas as pd
import language_tool_python
from tqdm import tqdm
import warnings
from chardet import detect

# Suppress warnings
warnings.filterwarnings("ignore")
torch.set_warn_always(False)

# Initialize LanguageTool
try:
    print("Initializing LanguageTool...")
    grammar_tool = language_tool_python.LanguageTool('en-US', config={'maxCheckThreads': 1})
    print("✓ LanguageTool initialized")
except Exception as e:
    print(f"× LanguageTool initialization failed: {e}")
    grammar_tool = None

# ================== CSV HANDLING FUNCTIONS ==================
def validate_csv(file_path):
    """Check if CSV is properly formatted"""
    try:
        with open(file_path, 'rb') as f:
            raw = f.read()
            encoding = detect(raw)['encoding']
        
        # Test read with pandas
        pd.read_csv(file_path, encoding=encoding).head()
        return True
    except Exception as e:
        print(f"CSV Validation Error: {str(e)}")
        return False

def repair_csv(input_path, output_path):
    """Fix common CSV issues"""
    try:
        # Detect encoding
        with open(input_path, 'rb') as f:
            raw = f.read()
            encoding = detect(raw)['encoding']
        
        # Read and clean
        with open(input_path, 'r', encoding=encoding, errors='replace') as f:
            content = f.read()
        
        # Remove BOM if exists
        if content.startswith('\ufeff'):
            content = content[1:]
        
        # Standardize line endings and quotes
        content = content.replace('\r\n', '\n').replace('"', "'")
        
        # Write cleaned file
        with open(output_path, 'w', encoding='utf-8') as f:
            f.write(content)
        
        print(f"Repaired CSV saved to {output_path}")
        return output_path
    except Exception as e:
        print(f"CSV Repair Failed: {str(e)}")
        return None

# ================== AUDIO PROCESSING FUNCTIONS ================== 
def preprocess_audio(input_path, output_path, target_sr=16000):
    try:
        y, sr = librosa.load(input_path, sr=None, mono=True)
        if sr != target_sr:
            y = librosa.resample(y, orig_sr=sr, target_sr=target_sr)
        y = librosa.util.normalize(y)
        sf.write(output_path, y, target_sr)
        return output_path
    except Exception as e:
        print(f"Audio preprocessing failed: {str(e)}")
        return None

# ================== MODEL FUNCTIONS ==================
WHISPER_MODELS = {}
def load_model(name="base"):
    if name not in WHISPER_MODELS:
        try:
            print(f"Loading Whisper model: {name}")
            model = whisper.load_model(name)
            device = 'cuda' if torch.cuda.is_available() else 'cpu'
            model = model.to(device)
            model.float()
            WHISPER_MODELS[name] = model
        except Exception as e:
            print(f"Model loading failed: {str(e)}")
            return None
    return WHISPER_MODELS[name]

def transcribe_audio(model, audio_path):
    try:
        with torch.inference_mode():
            torch.backends.cudnn.deterministic = True
            result = model.transcribe(
                audio_path,
                language="en",
                fp16=False,
                verbose=None
            )
        return result["text"].strip()
    except Exception as e:
        print(f"Transcription failed: {str(e)}")
        return None

# ================== GRAMMAR FUNCTIONS ==================
def check_grammar(text, max_retries=3):
    if grammar_tool is None or not text.strip():
        return []
    for attempt in range(max_retries):
        try:
            return grammar_tool.check(text)
        except Exception:
            if attempt == max_retries - 1:
                print("Grammar check failed after retries")
                return []
            continue

def extract_grammar_features(text, matches):
    try:
        words = max(len(text.split()), 1)
        total_errors = len(matches)
        errors_per_100 = (total_errors / words) * 100
        type_counts = {}
        for m in matches:
            error_type = getattr(m, 'ruleId', 'UNKNOWN').split('_')[0]
            type_counts[error_type] = type_counts.get(error_type, 0) + 1
        return {
            'total_errors': total_errors,
            'errors_per_100': errors_per_100,
            **{f'errors_{k}': v for k, v in type_counts.items()}
        }
    except Exception:
        return {'total_errors': 0, 'errors_per_100': 0}

# ================== MAIN PROCESSING ==================
def process_single_file(audio_path, audio_dir, model_name="base"):
    try:
        input_path = os.path.join(audio_dir, audio_path)
        proc_path = os.path.join("/tmp", f"proc_{audio_path}")
        
        if not os.path.exists(input_path):
            print(f"File not found: {input_path}")
            return None

        proc_path = preprocess_audio(input_path, proc_path)
        if not proc_path:
            return None

        model = load_model(model_name)
        if not model:
            return None
            
        transcript = transcribe_audio(model, proc_path)
        if not transcript:
            return None

        matches = check_grammar(transcript)
        features = extract_grammar_features(transcript, matches)

        try:
            os.remove(proc_path)
        except:
            pass

        return {
            'filename': audio_path,
            'transcript': transcript,
            'features': features
        }
    except Exception as e:
        print(f"Processing failed: {str(e)}")
        return None

# ================== MAIN EXECUTION ==================
def main():
    # Configuration
    input_dir = '/kaggle/input/shl-intern-hiring-assessment/dataset/audios_train'
    labels_csv = '/kaggle/input/shl-intern-hiring-assessment/dataset/train.csv'
    output_csv = '/kaggle/working/results.csv'
    
    # Validate and repair CSV if needed
    if not validate_csv(labels_csv):
        repaired_csv = '/kaggle/working/repaired_train.csv'
        if not repair_csv(labels_csv, repaired_csv):
            print("Critical: Failed to repair CSV")
            return
        labels_csv = repaired_csv
    
    # Load data with error handling
    try:
        labels_df = pd.read_csv(
            labels_csv,
            encoding='utf-8',
            quotechar="'",
            on_bad_lines='skip'
        )
        audio_files = labels_df['filename'].tolist()
    except Exception as e:
        print(f"Failed to load CSV: {str(e)}")
        return
    
    # Process files
    print(f"\nProcessing {len(audio_files)} files...")
    results = []
    for audio_file in tqdm(audio_files, desc="Processing"):
        result = process_single_file(audio_file, input_dir)
        if result:
            results.append(result)
    
    # Save results
    if results:
        features_df = pd.DataFrame([r['features'] for r in results])
        features_df['filename'] = [r['filename'] for r in results]
        
        try:
            features_df = features_df.merge(labels_df, on='filename', how='left')
            if 'label' in features_df.columns:
                X = features_df.drop(columns=['filename', 'label']).fillna(0)
                y = features_df['label']
                model = LinearRegression()
                model.fit(X, y)
                preds = np.clip(model.predict(X), 0, 5)
                print(f"\nModel Performance:")
                print(f"- MSE: {mean_squared_error(y, preds):.4f}")
                print(f"- MAE: {mean_absolute_error(y, preds):.4f}")
        except Exception as e:
            print(f"Merge/training failed: {str(e)}")
        
        try:
            pd.DataFrame(results).to_csv(output_csv, index=False)
            print(f"\nResults saved to {output_csv}")
        except Exception as e:
            print(f"Failed to save results: {str(e)}")
    else:
        print("No files processed successfully")

if __name__ == "__main__":
    main()

Initializing LanguageTool...


✓ LanguageTool initialized

Processing 444 files...


Processing:   0%|          | 0/444 [00:00<?, ?it/s]

Loading Whisper model: base



  0%|                                               | 0.00/139M [00:00<?, ?iB/s]
  0%|                                       | 192k/139M [00:00<01:26, 1.68MiB/s]
  0%|▏                                      | 656k/139M [00:00<00:43, 3.35MiB/s]
  1%|▎                                     | 1.25M/139M [00:00<00:30, 4.66MiB/s]
  1%|▍                                     | 1.71M/139M [00:00<00:31, 4.60MiB/s]
  2%|▋                                     | 2.39M/139M [00:00<00:26, 5.42MiB/s]
  2%|▊                                     | 3.05M/139M [00:00<00:24, 5.90MiB/s]
  3%|█                                     | 3.78M/139M [00:00<00:22, 6.35MiB/s]
  3%|█▏                                    | 4.39M/139M [00:00<00:23, 6.04MiB/s]
  4%|█▎                                    | 4.98M/139M [00:00<00:23, 5.98MiB/s]
  4%|█▌                                    | 5.55M/139M [00:01<00:27, 5.00MiB/s]
  4%|█▋                                    | 6.06M/139M [00:01<00:47, 2.91MiB/s]
  5%|█▊                    


Model Performance:
- MSE: 0.9610
- MAE: 0.8098

Results saved to /kaggle/working/results.csv


In [3]:
import os
import librosa
import numpy as np
import soundfile as sf
import whisper
import torch
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
import pandas as pd
import language_tool_python
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")
torch.set_warn_always(False)

# Initialize LanguageTool
try:
    print("Initializing LanguageTool...")
    grammar_tool = language_tool_python.LanguageTool('en-US')
    print("✓ LanguageTool initialized")
except Exception as e:
    print(f"× LanguageTool initialization failed: {e}")
    grammar_tool = None

# Audio Preprocessing with enhanced error handling
def preprocess_audio(input_path, output_path, target_sr=16000):
    try:
        # Load with explicit mono conversion and resampling
        y, sr = librosa.load(input_path, sr=None, mono=True)
        if sr != target_sr:
            y = librosa.resample(y, orig_sr=sr, target_sr=target_sr)
        y = librosa.util.normalize(y)
        sf.write(output_path, y, target_sr)
        return output_path
    except Exception as e:
        print(f"Audio preprocessing failed for {os.path.basename(input_path)}")
        return None

# Whisper Model Loading with CUDA/CPU management
WHISPER_MODELS = {}
def load_model(name="base"):
    if name not in WHISPER_MODELS:
        try:
            print(f"Loading Whisper model: {name}")
            model = whisper.load_model(name)
            
            # Handle CUDA/CPU automatically
            device = 'cuda' if torch.cuda.is_available() else 'cpu'
            model = model.to(device)
            
            # Force FP32 and disable half-precision
            model.float()
            WHISPER_MODELS[name] = model
        except Exception as e:
            print(f"Failed to load Whisper model: {e}")
            return None
    return WHISPER_MODELS[name]

# Robust transcription with tensor shape handling
def transcribe_audio(model, audio_path):
    try:
        # Disable half-precision and enable deterministic algorithms
        with torch.inference_mode():
            torch.backends.cudnn.deterministic = True
            result = model.transcribe(
                audio_path,
                language="en",
                fp16=False,  # Critical for stability
                verbose=None  # Disable progress bars
            )
        return result["text"].strip()
    except RuntimeError as e:
        if "reshape" in str(e):
            print(f"Skipping {os.path.basename(audio_path)} due to tensor shape issue")
        else:
            print(f"Transcription failed for {os.path.basename(audio_path)}")
        return None
    except Exception as e:
        print(f"Unexpected error processing {os.path.basename(audio_path)}")
        return None

# Grammar checking with retries
def check_grammar(text, max_retries=3):
    if grammar_tool is None or not text.strip():
        return []
    
    for attempt in range(max_retries):
        try:
            return grammar_tool.check(text)
        except Exception:
            if attempt == max_retries - 1:
                print("Grammar check failed after retries")
                return []
            continue

# Feature extraction with fallback
def extract_grammar_features(text, matches):
    try:
        words = max(len(text.split()), 1)
        total_errors = len(matches)
        errors_per_100 = (total_errors / words) * 100
        
        type_counts = {}
        for m in matches:
            error_type = getattr(m, 'ruleId', 'UNKNOWN').split('_')[0]
            type_counts[error_type] = type_counts.get(error_type, 0) + 1
        
        return {
            'total_errors': total_errors,
            'errors_per_100': errors_per_100,
            **{f'errors_{k}': v for k, v in type_counts.items()}
        }
    except Exception:
        return {
            'total_errors': 0,
            'errors_per_100': 0
        }

# Main processing pipeline
def process_single_file(audio_path, audio_dir, model_name="base"):
    try:
        # Preprocessing
        proc_path = preprocess_audio(
            os.path.join(audio_dir, audio_path),
            os.path.join("/tmp", f"proc_{audio_path}")
        )
        if proc_path is None:
            return None

        # Transcription
        model = load_model(model_name)
        if model is None:
            return None
            
        transcript = transcribe_audio(model, proc_path)
        if not transcript:
            return None

        # Grammar analysis
        matches = check_grammar(transcript)
        features = extract_grammar_features(transcript, matches)

        return {
            'filename': audio_path,
            'transcript': transcript,
            'features': features
        }

    except Exception:
        return None

# Main execution
def main():
    # Configuration
    input_dir = '/kaggle/input/shl-intern-hiring-assessment/dataset/audios_train'
    labels_csv = '/kaggle/input/shl-intern-hiring-assessment/dataset/train.csv'
    output_csv = '/kaggle/working/results.csv'
    
    # Load data
    labels_df = pd.read_csv(labels_csv)
    audio_files = labels_df['filename'].tolist()
    
    # Process files with single worker for maximum stability
    print(f"\nProcessing {len(audio_files)} files...")
    results = []
    
    for audio_file in tqdm(audio_files, desc="Processing"):
        result = process_single_file(audio_file, input_dir)
        if result:
            results.append(result)
    
    print(f"\nCompleted: {len(results)}/{len(audio_files)} files processed")
    
    # Prepare DataFrame
    if results:
        features_df = pd.DataFrame([r['features'] for r in results])
        features_df['filename'] = [r['filename'] for r in results]
        features_df = features_df.merge(labels_df, on='filename', how='left')
        
        # Train model if labels exist
        if 'label' in features_df.columns:
            X = features_df.drop(columns=['filename', 'label']).fillna(0)
            y = features_df['label']
            
            model = LinearRegression()
            model.fit(X, y)
            
            # Evaluate
            preds = np.clip(model.predict(X), 0, 5)
            print(f"\nModel Performance:")
            print(f"- MSE: {mean_squared_error(y, preds):.4f}")
            print(f"- MAE: {mean_absolute_error(y, preds):.4f}")
        
        # Save results
        pd.DataFrame(results).to_csv(output_csv, index=False)
        print(f"\nResults saved to {output_csv}")
    else:
        print("No files processed successfully")

if __name__ == "__main__":
    main()

Initializing LanguageTool...


✓ LanguageTool initialized

Processing 444 files...


Processing:   0%|          | 0/444 [00:00<?, ?it/s]

Loading Whisper model: base



  0%|                                               | 0.00/139M [00:00<?, ?iB/s]
  6%|██▏                                   | 7.84M/139M [00:00<00:01, 82.2MiB/s]
 21%|████████▏                              | 29.1M/139M [00:00<00:00, 165MiB/s]
 32%|████████████▌                          | 44.8M/139M [00:00<00:00, 165MiB/s]
 44%|█████████████████                      | 60.6M/139M [00:00<00:00, 156MiB/s]
 55%|█████████████████████▎                 | 75.6M/139M [00:00<00:00, 156MiB/s]
 65%|█████████████████████████▍             | 90.5M/139M [00:00<00:00, 155MiB/s]
 76%|██████████████████████████████▍         | 105M/139M [00:00<00:00, 126MiB/s]
 85%|█████████████████████████████████▎     | 118M/139M [00:01<00:00, 74.6MiB/s]
 92%|████████████████████████████████████   | 128M/139M [00:01<00:00, 69.2MiB/s]
100%|████████████████████████████████████████| 139M/139M [00:01<00:00, 100MiB/s]
Processing: 100%|██████████| 444/444 [17:20<00:00,  2.34s/it]



Completed: 443/444 files processed

Model Performance:
- MSE: 0.9651
- MAE: 0.8047

Results saved to /kaggle/working/results.csv


In [3]:
!pip install language-tool-python